In [1]:
rm( list=ls() )  #Borro todos los objetos
gc()   #Garbage Collection

library("data.table")
library("rpart")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,614669,32.9,1321401,70.6,1321401,70.6
Vcells,1093272,8.4,8388608,64.0,1632451,12.5


Warning message:
"package 'data.table' was built under R version 4.2.3"
Warning message:
"package 'rpart' was built under R version 4.2.3"


In [2]:
# particionar agrega una columna llamada fold (por defecto) a un dataset
# que consiste en una particion estratificada segun agrupa
# particionar(  data=dataset
              #,division=c(70,30)
              #, agrupa=clase_ternaria
              #, seed=semilla) crea una particion 70, 30 

particionar  <- function(
  data #el dataset sobre el que se trabaja, va a tener una nueva columna
  ,division #este parametro determina las proporciones, vector de N cantidad
  ,agrupa="" #aca va el criterio de agrupacion
  ,campo="fold" #el nombre del nuevo campo, default "fold"
  ,start=1 #los numeros asignados parten de 1 por defecto
  ,seed=NA) #seed default NA
{
  if(!is.na(seed)) set.seed(seed)

  bloque <- unlist(
    mapply(
      function(x, y) {rep(y, x)}
      ,division
      ,seq(from = start
        ,length.out = length(division)
      )
    )
  )
  #esto ayuda a visualizar lo que hace la funcion

  #modelo de como es un bloque
  #walA = bloque 
  #el modelo repetido segun la cantidad de rows
  #walB = rep( bloque, ceiling(nrow(data)/length(bloque))) 
  #la muestra de ese modelo que ahora lleva otro orden
  #walC = sample( walB)
  #preparo el retorno
  #listReturn <- list(walA,walB,walC)
  data[ , (campo) :=  sample( rep( bloque, ceiling(.N/length(bloque))))[1:.N], by= agrupa ]
  
  #return(listReturn)
}

In [3]:
#apunto el working directory al directorio donde esta ubicado el script
setwd(".\\") 
getwd() #solo para validar

[1] "d:/Onedrive/Austral/Cursos/08 - Laboratorio de Implementacion 1/labo2023v/src/rpart"

In [4]:
#cargo los datos
dataset  <- fread("./datasets/dataset_pequeno.csv")
#cat("Lineas en el archivo original: ", nrow(dataset), "\n")

#trabajo solo con los datos con clase, es decir 202107
dataset  <- dataset[ clase_ternaria!= "" ]
#cat("Lineas con clase: ", nrow(dataset), "\n")

In [5]:
#particiono estratificadamente el dataset
#Cambiar por la primer semilla de cada uno!
a = particionar( data=dataset, division=c(7,3), agrupa="clase_ternaria", seed= 262139 )  #Cambiar por la primer semilla de cada uno !

In [6]:
#cat("Modelo Bloque: ", unlist(a[1]), "\n")
#cat("Bloque Repetido: ", unlist(unlist(a[2])[1:40]), "...\n")
#cat("Sample: ", unlist(unlist(a[3])[1:40]), "...\n")

In [16]:
par_cp=         seq(-1, 0, by = 0.25)     #complejidad minima
par_minsplit=   seq(400,2000, by = 400)               #minima cantidad de registros en un nodo para hacer el split
par_minbucket=  c(10,50,100,250,500)                #minima cantidad de registros en una hoja
par_maxdepth=   c(4,6,8,10,12)                 #profundidad máxima del arbol

[1] -1.00 -0.75 -0.50 -0.25  0.00

[1]  400  800 1200 1600 2000

[1]  10  50 100 250 500

[1]  4  6  8 10 12

In [88]:
colCp = list()
colMs = list()
colMb = list()
colMd = list()

for (i in 1:5){
    for (j in 1:5){
        for (k in 1:5){
            for (l in 1:5){
                colCp = c(colCp,par_cp[i])
                colMs = c(colMs,par_minsplit[j])
                colMb = c(colMb,par_minbucket[k])
                colMd = c(colMd,par_maxdepth[l])
                #print(paste(par_cp[i], par_minsplit[j], par_minsplit[k], par_minsplit[l]))
            }
        }
    }
}
configs = data.table("Cp" = colCp, "Ms" = colMs, "Mb" = colMb, "Md" = colMd)
head(configs)

Cp,Ms,Mb,Md
<list>,<list>,<list>,<list>
-1,400,10,4
-1,400,10,6
-1,400,10,8
-1,400,10,10
-1,400,10,12
-1,400,50,4


In [82]:
fin_TestingTotal = list()
fin_TestingBaja2 = list()
fin_Estimulos = list()
fin_Aciertos = list()
fin_Ganancia = list()

for (testConfig in 1:nrow(configs)){
    param_basicos  <- list( "cp"=           configs$Cp[testConfig],  #complejidad minima
                            "minsplit"=     configs$Ms[testConfig],  #minima cantidad de registros en un nodo para hacer el split
                            "minbucket"=    configs$Mb[testConfig],  #minima cantidad de registros en una hoja
                            "maxdepth"=     configs$Md[testConfig] ) #profundidad máxima del arbol
    #genero el modelo
    modelo  <- rpart("clase_ternaria ~ .",     #quiero predecir clase_ternaria a partir del resto
                    data= dataset[ fold==1],  #fold==1  es training,  el 70% de los datos
                    xval= 0,
                    control=  param_basicos )  #aqui van los parametros

    #aplico el modelo a los datos de testing
    prediccion  <- predict( modelo,   #el modelo que genere recien
                            dataset[ fold==2],  #fold==2  es testing, el 30% de los datos
                            type= "prob") #type= "prob"  es que devuelva la probabilidad

    #prediccion es una matriz con TRES columnas, llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
    #cada columna es el vector de probabilidades 

    #agrego una columna que es la de las ganancias
    dataset[  , ganancia :=  ifelse( clase_ternaria=="BAJA+2", 117000, -3000 ) ]
    #para testing agrego la probabilidad
    dataset[ fold==2 , prob_baja2 := prediccion[, "BAJA+2"] ]
    #calculo la ganancia en testing  qu es fold==2
    ganancia_test  <- dataset[ fold==2 & prob_baja2 >  0.025, sum(ganancia) ]
    #escalo la ganancia como si fuera todo el dataset
    ganancia_test_normalizada  <-  ganancia_test / 0.3
    estimulos  <- dataset[ fold==2 & prob_baja2 > 0.025 , .N ]
    aciertos   <- dataset[ fold==2 & prob_baja2 > 0.025 & clase_ternaria =="BAJA+2", .N ]
    
    fin_TestingTotal = c(fin_TestingTotal,dataset[fold==2, .N])
    fin_TestingBaja2 = c(fin_TestingBaja2,dataset[ fold==2 & clase_ternaria =="BAJA+2", .N ]) 
    fin_Estimulos = c(fin_Estimulos,estimulos)
    fin_Aciertos = c(fin_Aciertos,aciertos)
    fin_Ganancia = c(fin_Ganancia,ganancia_test_normalizada)
}


In [83]:
finalResults = data.table(
    "ComplejidadMinima" = colCp
    , "MinSplit" = colMs
    , "MinBucket" = colMb
    , "MaxDepth" = colMd
    , "TestingTotal" = fin_TestingTotal
    , "TesingBaja2" = fin_TestingBaja2
    , "Estimulos" = fin_Estimulos
    , "Aciertos" = fin_Aciertos
    , "Ganancia" = fin_Ganancia)

In [85]:
dir.create( "./exp/" )

#fwrite( dapply[ , list(numero_de_cliente, Predicted) ], #solo los campos para Kaggle
fwrite( finalResults, #solo los campos para Kaggle
        file= "./exp/treeResults.csv",
        sep=  "," )


Warning message in dir.create("./exp/"):
"'.\exp' already exists"
